# seedbleniding

exp40,exp49 のseed42,614


In [2]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from itertools import combinations

import pickle

In [3]:
train_42 = pd.read_csv("../output/exp52_xgb_lagdiff_c3_statedate/oof_exp52_xgb_0.7951738236320218_5fold_seed42.csv")
test_42 = pd.read_csv("../output/exp52_xgb_lagdiff_c3_statedate/test_exp52_xgb_0.7951738236320218_5fold_2_seed42.csv")

train_614 = pd.read_csv("../output/exp52_614_xgb_lagdiff_c3_statedate/oof_614_exp52_xgb_0.020605391255782844_5fold_seed614.csv")
test_614 = pd.read_csv("../output/exp52_614_xgb_lagdiff_c3_statedate/test_614_exp52_xgb_0.020605391255782844_5fold_seed614.csv")



In [13]:
test_42.head()

,customer_ID,exp52_xgb_sub
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.029535
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000827
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.031328
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.238987
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.879322


In [6]:
test_614.head()

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.028106
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000828
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.032835
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.229212
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.883701


In [14]:
oof = train_42.merge(train_614,on = "customer_ID")
sub = test_42.merge(test_614,on = "customer_ID")


In [15]:
oof["exp52_42+614_oof"] = (oof["exp52_xgb_oof_x"] + oof["exp52_xgb_oof_y"])/2
sub["prediction"] = (sub["prediction"] + sub["exp52_xgb_sub"])/2 

In [16]:
sub

,customer_ID,exp52_xgb_sub,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.029535,0.028820
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.000827,0.000827
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.031328,0.032082
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.238987,0.234099
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.879322,0.881512
...,...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.014330,0.014196
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.837867,0.835135
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.441129,0.453263
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.279516,0.275847


In [17]:
oof

,customer_ID,target_x,exp52_xgb_oof_x,target_y,exp52_xgb_oof_y,exp52_42+614_oof
0,0001812036f1558332e5c0880ecbad70b13a6f28ab04a8...,1,0.776882,1,0.778236,0.777559
1,00031e8be98bc3411f6037cbd4d3eeaf24b3ae221682b7...,0,0.010273,0,0.008497,0.009385
2,0004860c260168fcaad0734a1dfedb7ceb1a83aaac54e2...,1,0.997708,1,0.997463,0.997586
3,0004b8596c4946866d1b27a8b40488ecf49c6eae9c7bf4...,0,0.002184,0,0.001786,0.001985
4,000548e99fa24cef8377e68e602e4bd70d30500a007999...,0,0.000925,0,0.001113,0.001019
...,...,...,...,...,...,...
458908,fff908439f19e2b2a4eb26432d51d8e38a0b757515f0f3...,0,0.022977,0,0.017467,0.020222
458909,fffcb68b6284b022106c4a2726d7d1cee2a3b774f2e8da...,0,0.005355,0,0.003160,0.004257
458910,fffcfd4dcb5311e3b7022eeec6c194f0ee01528bb45a01...,0,0.002899,0,0.002191,0.002545
458911,fffdc0cf228085b4275b38ebe6eb915766af3fecb2ae28...,0,0.000331,0,0.000197,0.000264


In [19]:
oof[["customer_ID","exp52_42+614_oof"]].to_csv("exp52_oof_xgb_0.795606682_seedblend.csv",index = False)
sub[["customer_ID","prediction"]].to_csv("exp52_sub_xgb_0.795606682_seedblend.csv",index = False)


In [18]:
def amex_metric(y_true, y_pred):
    labels = np.transpose(np.array([y_true, y_pred]))
    labels = labels[labels[:, 1].argsort()[::-1]]
    weights = np.where(labels[:,0]==0, 20, 1)
    cut_vals = labels[np.cumsum(weights) <= int(0.04 * np.sum(weights))]
    top_four = np.sum(cut_vals[:,0]) / np.sum(labels[:,0])
    gini = [0,0]
    for i in [1,0]:
        labels = np.transpose(np.array([y_true, y_pred]))
        labels = labels[labels[:, i].argsort()[::-1]]
        weight = np.where(labels[:,0]==0, 20, 1)
        weight_random = np.cumsum(weight / np.sum(weight))
        total_pos = np.sum(labels[:, 0] *  weight)
        cum_pos_found = np.cumsum(labels[:, 0] * weight)
        lorentz = cum_pos_found / total_pos
        gini[i] = np.sum((lorentz - weight_random) * weight)
    return 0.5 * (gini[1]/gini[0] + top_four)


amex_metric(oof["target_x"], oof["exp52_42+614_oof"])

0.7956066822598196